In [1]:
import numpy as np
import tensorflow as tf

batch_size = 100
V = 10000
D = 200
C = 3

y_c = tf.placeholder(tf.int32, shape=(batch_size, C))
E = tf.Variable(tf.random_uniform((V, D), -1.0, 1.0)) # 乱数の設定を論文から参照する
tilde_y_c = tf.nn.embedding_lookup(E, y_c) 

Help on function initialize_all_variables in module tensorflow.python.ops.variables:

initialize_all_variables()
    Returns an Op that initializes all variables.
    
    This is just a shortcut for `initialize_variables(all_variables())`
    
    Returns:
      An Op that initializes all variables in the graph.



In [2]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    #feed_dict={y_c: np.arange(batch_size)}
    feed_dict={y_c: np.arange(batch_size*C).reshape(batch_size, C)}
    print(sess.run(tilde_y_c, feed_dict=feed_dict).shape)

(100, 3, 200)


In [3]:
length = 100
x = tf.placeholder(tf.int32, shape=(batch_size, None))
y = x * 3
sess = tf.Session()
feed_dict = {x: np.arange(batch_size*length).reshape(batch_size, length)}
output = sess.run(y, feed_dict=feed_dict)


In [4]:
import numpy as np
import tensorflow as tf

batch_size = 100
V = 10000
D = 200

length = 60
x = tf.placeholder(tf.int32, shape=(batch_size, None))
x = tf.Print(x, [x])
F = tf.Variable(tf.random_uniform([V, D], -1.0, 1.0)) # 乱数の設定を参照
tilde_x = tf.nn.embedding_lookup(F, x)
sess = tf.Session()
sess.run(tf.initialize_all_variables())
feed_dict = {x: np.arange(batch_size*length).reshape(batch_size, length)}
#feed_dict = {x: np.arange(batch_size)}
output = sess.run(tilde_x, feed_dict=feed_dict)
print(output.shape)

(100, 60, 200)


In [123]:
import numpy as np
import tensorflow as tf

batch_size = 50
V = 10000
D = 200

length = 6 # ここでは、length大きくしすぎると単語数がVより大きくなるので注意
x = tf.placeholder(tf.int32, shape=(batch_size, None))
F = tf.Variable(tf.random_uniform([V, D], -1.0, 1.0)) # 乱数の設定を参照
tilde_x = tf.cast(tf.nn.embedding_lookup(F, x), tf.float32)
p = tf.placeholder(tf.float32, shape=[batch_size, None])
print('--------------------------------------------------')
print('tilde_x: ', tilde_x.get_shape())
print('slice: ', tf.slice(tilde_x, [0, 0, 0], [1, -1, -1]).get_shape())
print('slice: ', tf.slice(tilde_x, [1, 0, 0], [1, -1, -1]).get_shape())
print('--------------------------------------------------')
print('p:', p.get_shape())
print('slice: ', tf.slice(p, [0, 0], [1, -1]).get_shape())
print('slice: ', tf.slice(p, [3, 0], [1, -1]).get_shape())
print('--------------------------------------------------')
for i in range(batch_size):
    if i == 0:
        enc = tf.matmul(tf.slice(p, [i, 0], [1, -1]), tf.reshape(tf.slice(tilde_x, [i, 0, 0], [1, -1, -1]), [-1, D]))
    else:
        temp = tf.matmul(tf.slice(p, [i, 0], [1, -1]), tf.reshape(tf.slice(tilde_x, [i, 0, 0], [1, -1, -1]), [-1, D]))
        enc = tf.concat(0, [enc, temp])
print('enc: ', enc.get_shape())
sess = tf.Session()
sess.run(tf.initialize_all_variables())
feed_dict = {x: np.arange(batch_size*length).reshape(batch_size, length), p: (np.ones((batch_size, length))/length).astype(np.float32)}
output = sess.run(enc, feed_dict=feed_dict)
print(output.shape)

--------------------------------------------------
tilde_x:  TensorShape([Dimension(50), Dimension(None), Dimension(200)])
slice:  TensorShape([Dimension(1), Dimension(None), Dimension(200)])
slice:  TensorShape([Dimension(1), Dimension(None), Dimension(200)])
--------------------------------------------------
p: TensorShape([Dimension(50), Dimension(None)])
slice:  TensorShape([Dimension(1), Dimension(None)])
slice:  TensorShape([Dimension(1), Dimension(None)])
--------------------------------------------------
enc:  TensorShape([Dimension(50), Dimension(200)])
(50, 200)


In [125]:
help(tf.nn.embedding_lookup)

Help on function embedding_lookup in module tensorflow.python.ops.embedding_ops:

embedding_lookup(params, ids, partition_strategy='mod', name=None)
    Looks up `ids` in a list of embedding tensors.
    
    This function is used to perform parallel lookups on the list of
    tensors in `params`.  It is a generalization of
    [`tf.gather()`](../../api_docs/python/array_ops.md#gather), where `params` is
    interpreted as a partition of a larger embedding tensor.
    
    If `len(params) > 1`, each element `id` of `ids` is partitioned between
    the elements of `params` according to the `partition_strategy`.
    In all strategies, if the id space does not evenly divide the number of
    partitions, each of the first `(max_id + 1) % len(params)` partitions will
    be assigned one more id.
    
    If `partition_strategy` is `"mod"`, we assign each id to partition
    `p = id % len(params)`. For instance,
    13 ids are split across 5 partitions as:
    `[[0, 5, 10], [1, 6, 11], [2, 7

In [127]:
t = np.array([3, 1, 6])
t_onehot = np.zeros((3, 10))
t_onehot[np.arange(3), t] = 1
t_onehot 

array([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.]])

In [129]:
print(F.get_shape())
print(tf.argmax(F, 1).get_shape())

TensorShape([Dimension(10000), Dimension(200)])
TensorShape([Dimension(10000)])


In [42]:
import re
import pandas as pd
dataset_path = '../../data/reuters/Business/year=2016/reuters_201611.csv'
dataset = pd.read_csv(dataset_path, usecols=[5, 6])

def first_sentence(sentences):
    m = re.search('-(.*?)。', sentences)
    return m.group(1)

In [44]:
index = 560
print('----------------- title -----------------')
print(dataset.ix[index, 'title'])
print()
print('----------------- content -----------------')
print(dataset.ix[index, 'content'])
print()
print('----------------- first sentence -----------------')
print(first_sentence(dataset.ix[index, 'content']))

----------------- title -----------------
アングル：高まる欧州選挙リスク、英米の結果で身構える市場

----------------- content -----------------
［ロンドン １１日 ロイター］ - 英国民投票での欧州連合（ＥＵ）離脱派の勝利、米大統領選でのトランプ氏勝利──。既得権益層に対する大衆の反発を２度も見せつけられた金融市場は、来年に向け欧州で予定される国民投票や選挙で同様の事態が起こる可能性を覚悟し始めている。来年はオランダ、フランス、ドイツで選挙が予定されており、場合によってはイタリアと英国でも選挙が行われるかもしれない。当面の試金石となるのが、イタリアで１２月４日に予定される憲法改正の是非を問う国民投票。同じ日にオーストリアでは大統領選挙の決選投票のやり直しが実施され、候補者２人のうち１人は極右の人物だ。トランプ氏の財政拡張路線がインフレを招くとの見方から、欧米の国債相場はここ数日で全般に下落したが、中でもフランスとイタリアの下げ幅が大きいのも無理はない。ドイツの１０年国債利回りは１１日の週に１７ベーシスポイント（ｂｐ）程度上昇。これに対し、フランスとイタリアの１０年国債利回りは政治リスクも織り込んで約２５ｂｐ上昇している。フランス１０年国債利回りは０．７２％と、１月以来の高水準に迫り、ドイツとのスプレッドは４４ｂｐと、昨年７月以来で最大となった。ＩＮＧのシニア金利ストラテジスト、Martin Van Vliet氏は「英国の投票でアンチ既得権益層が勝ち、米国でも同じ結果が出たので、だれもが『来年はユーロ圏のどこで選挙があるのだろう』と調べ始めた。そう、フランスだ。こうした国々が危ないかもしれないと見て、市場はズームインし始めた」と話す。フランス大統領選の仕組みを踏まえると、極右政党・国民戦線（ＦＮ）のマリーヌ・ルペン党首が最終的に勝利する可能性は小さいと見られているが、世論調査によると、第１回投票での得票数はルペン氏が最多となりそうだ。しかも米英の投票が予想外の結果だったため、市場はもう世論調査を鵜呑みにしなくなるだろう。＜株や為替も＞動揺は債券市場以外にも広がっている。株式市場では、トランプ氏勝利による「リフレ」相場がポピュリズム（大衆迎合主義）や保護主義の恐れを覆い隠しているが、

In [46]:
help(pd.core.groupby.DataFrameGroupBy)

Help on class DataFrameGroupBy in module pandas.core.groupby:

class DataFrameGroupBy(NDFrameGroupBy)
 |  Class for grouping and aggregating relational data. See aggregate,
 |  transform, and apply functions on this object.
 |  
 |  It's easiest to use obj.groupby(...) to use GroupBy, but you can also do:
 |  
 |  ::
 |  
 |      grouped = groupby(obj, ...)
 |  
 |  Parameters
 |  ----------
 |  obj : pandas object
 |  axis : int, default 0
 |  level : int, default None
 |      Level of MultiIndex
 |  groupings : list of Grouping objects
 |      Most users should ignore this
 |  exclusions : array-like, optional
 |      List of columns to exclude
 |  name : string
 |      Most users should ignore this
 |  
 |  Notes
 |  -----
 |  After grouping, see aggregate, apply, and transform functions. Here are
 |  some other brief notes about usage. When grouping by multiple groups, the
 |  result index will be a MultiIndex (hierarchical) by default.
 |  
 |  Iteration produces (key, group) tupl

In [65]:
import numpy as np
N = 10
a = np.arange(N)
batchsize = 3
n_batch = int((N-1)/batchsize + 1)
for i in range(n_batch):
    print(a[i*batchsize: (i+1)*batchsize])

[0 1 2]
[3 4 5]
[6 7 8]
[9]


In [11]:
import numpy as np
import pandas as pd
print(pd.DataFrame(np.arange(10).reshape(5, 2)))
for v in pd.DataFrame(np.arange(10).reshape(5, 2)).iterrows():
    print(type(v))
    print(len(v))
    print(v)

   0  1
0  0  1
1  2  3
2  4  5
3  6  7
4  8  9
<class 'tuple'>
2
(0, 0    0
1    1
Name: 0, dtype: int64)
<class 'tuple'>
2
(1, 0    2
1    3
Name: 1, dtype: int64)
<class 'tuple'>
2
(2, 0    4
1    5
Name: 2, dtype: int64)
<class 'tuple'>
2
(3, 0    6
1    7
Name: 3, dtype: int64)
<class 'tuple'>
2
(4, 0    8
1    9
Name: 4, dtype: int64)


In [21]:
a = [[[1, 2, 3], [4, 5], 3], [[2, 3, 4], [5, 6], 4]]


[[2, 3, 4], [5, 6], 4]

In [24]:
np.array([np.array([1,3,5]).astype(np.int32), np.array([3,4,6]).astype(np.int32)]).shape

(2, 3)

In [2]:
help(tf.floldl)

AttributeError: module 'tensorflow' has no attribute 'floldl'

In [4]:
import tensorflow as tf
print(tf.__version__)
help(tf.while_loop)

0.6.0


AttributeError: module 'tensorflow' has no attribute 'while_loop'

In [7]:
list(range(-1, 1+1))

[-1, 0, 1]